In [7]:
import pandas as pd, numpy as np, os
from sklearn.preprocessing import MinMaxScaler
os.chdir('/Users/walu/mdwarfs/ae_classifier')

In [4]:
df = pd.read_csv('../training_data/balona_lc_flux_1024p_256runwd.csv')
df = df[df['label'] != 'MIX']


OSError: Cannot save file into a non-existent directory: '../trainining_data'

In [5]:
df.to_csv('../training_data/balona_lc_flux_1024p_256runwd.csv')

In [11]:
all_existing_tic = pd.read_csv("../training_data/SOLR_NonSOLR_unbalanced.csv")['TIC']
gaia_balona_extended = pd.read_csv("../training_data/gaia_balona_eb_new_class.csv")
gaia_balona_extended = gaia_balona_extended[gaia_balona_extended['label'] != 'gaiadr3']
# Filter out entries that are not in the all_existing_tic
gaia_balona_extended = gaia_balona_extended[~gaia_balona_extended['TIC'].isin(all_existing_tic)]
gaia_balona_extended.shape


(37827, 1027)

In [14]:
gaia_balona_extended.drop(columns=['Unnamed: 0'], inplace=True)

In [15]:
gaia_balona_extended.head()

,0,1,2,3,4,5,6,7,8,9,...,1016,1017,1018,1019,1020,1021,1022,1023,TIC,label
9,0.208004,0.213319,0.222637,0.236645,0.256044,0.280046,0.302071,0.303227,0.258679,0.181512,...,0.041645,0.057917,0.084610,0.118263,0.150986,0.175395,0.186128,0.180598,100011351,NonSOLR
25,0.024028,0.024041,0.023925,0.023234,0.020689,0.014454,0.005960,0.001199,0.005327,0.008435,...,0.066209,0.077067,0.076954,0.069246,0.063047,0.062150,0.059008,0.045306,100101337,NonSOLR
151,0.006673,0.008730,0.009746,0.006102,0.001820,0.012080,0.020983,0.024279,0.018945,0.005115,...,0.004220,0.010115,0.014240,0.015842,0.014874,0.011932,0.007993,0.004073,1008153,NonSOLR
177,0.046413,0.049302,0.055201,0.065960,0.083600,0.105476,0.119569,0.108040,0.061942,0.010189,...,0.079494,0.064287,0.044481,0.021203,0.053388,0.108453,0.152933,0.172611,100918613,NonSOLR
182,0.038492,0.100343,0.140916,0.131648,0.079264,0.032107,0.090465,0.142421,0.157472,0.141705,...,0.125154,0.022800,0.152258,0.244663,0.265511,0.215090,0.140747,0.158146,100975890,NonSOLR


In [5]:
#min max scaling solr osc data
##solr_df = pd.read_csv("../training_data/solr_oscillators_all_sector.csv")  # Load the solr_df if needed
#import pandas as pd
#import numpy as np
#from sklearn.preprocessing import MinMaxScaler

def rowwise_minmax_scaling(df):
    # Separate signal, TIC, and label
    signal_data = df.iloc[:, :1024].values
    tic_column = df.iloc[:, 1024].values
    label_column = df.iloc[:, 1025].values

    # Row-wise MinMaxScaler with range (0, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_signals = np.array([scaler.fit_transform(row.reshape(-1, 1)).flatten() for row in signal_data])

    # Combine everything back into a DataFrame
    scaled_df = pd.DataFrame(scaled_signals)
    scaled_df['TIC'] = tic_column
    scaled_df['label'] = label_column

    return scaled_df
#scaled_solr_df = rowwise_minmax_scaling(solr_df)
#scaled_solr_df.to_csv("../training_data/solr_oscillators_all_sector_scaled.csv", index=False)

In [9]:
scaled_solr_df.to_csv("../training_data/solr_oscillators_all_sector.csv", index=False)

In [16]:

# Load your original DataFrame (assumed to be called df)
df = pd.read_csv("../training_data/PULS_based_balanced_sample.csv")  # Uncomment and update if needed
solr_df = pd.read_csv("../training_data/solr_oscillators_all_sector.csv")  # Load the solr_df if needed
solr_df.columns = list(range(1024)) + solr_df.columns[1024:].tolist()


# 1. Keep all 'PULS' rows
df_puls = df[df['label'] == 'PULS']
df_puls['label'] = 'NonSOLR'

# 2. Keep all 'EB' rows, but relabel to 'NonPULS'
df_eb = df[df['label'] == 'EB'].copy()
df_eb['label'] = 'NonSOLR'

# 3. Randomly sample 6000 'ROT' rows and relabel
df_rot = df[df['label'] == 'ROT'].copy()
df_rot['label'] = 'NonSOLR'

# 4. Randomly sample 6000 'NV' rows and relabel
df_nv = df[df['label'] == 'NV'].copy()
df_nv['label'] = 'NonSOLR'

gaia_balona_extended['label'] = 'NonSOLR'

expected_cols = list(range(1024)) + ['TIC', 'label']

for df_sub in [df_puls, df_eb, df_rot, df_nv, solr_df, gaia_balona_extended]:
    df_sub.columns = expected_cols



# Combine all into a new DataFrame
df_new = pd.concat([df_puls, df_eb, df_rot, df_nv, gaia_balona_extended, solr_df], ignore_index=True)

# Optional: shuffle the result
##df_new = df_new.sample(frac=1, random_state=42).reset_index(drop=True)

# Save to a new CSV file


/var/folders/3k/qhhyzp650rsb_v_vhzr1qzm80000gn/T/ipykernel_64102/2838218192.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_puls['label'] = 'NonSOLR'


In [20]:
df_new.shape

(116165, 1026)

In [19]:
df_new.to_csv("../training_data/SOLR_NonSOLR_unbalanced_v2.csv", index=False)


In [8]:
new_pure = pd.read_csv('../pulsator_sample_prep/new_pure_pulsator_1024periodogram.csv')
df = pd.DataFrame({
    'TIC' : new_pure['TIC'],
    'label' : new_pure['label'],
})
df.to_csv('../pulsator_sample_prep/new_pure_pulsator_catalog.csv', index=False)

In [5]:
#Extracting TIC and labels of  balanced pulsator catalog
#Intended to be used for downloading full periodogram with binned shape
df_balanced = pd.read_csv('../training_data/PULS_based_balanced_sample.csv')
new_df = pd.DataFrame(
    {'TIC': df_balanced['TIC'],
     'label': df_balanced['label']
    }
)
new_df.to_csv('../pulsator_based_balanced_catalog.csv', index=False)

In [27]:
#prepping light curve dataset
#First, non solar light curves
non_solar_df = pd.read_csv('../training_data/1024_lc_flux_dataset_non_solar.csv')
solar_df = pd.read_csv('../training_data/solar_lc_1024.csv')


In [29]:
non_solar_df['label'] = 'NonSOLR'
solar_df['label'] = 'SOLR'
# Combine both DataFrames
combined_df = pd.concat([non_solar_df, solar_df], ignore_index=True)

In [34]:
combined_df.to_csv('../training_data/SOLR_NonSOLR_unbalanced_lc.csv', index=False)

In [3]:
combined_df = pd.read_csv('../training_data/SOLR_NonSOLR_unbalanced_lc.csv')

In [8]:
normalized_combined_df = rowwise_minmax_scaling(combined_df)

In [15]:
normalized_combined_df.to_csv('../training_data/SOLR_NonSOLR_unbalanced_lc.csv', index=False)

In [21]:
#copying files from rawfiles_periodogram to training_data_dir
import shutil

source_dir = "/Volumes/Seagate Por/rawfiles_periodogram"
destination_dir = "/Volumes/Seagate Por/SURF25/train_data_dir/rawfiles_periodogram"
tics = pd.read_csv('../training_data/SOLR_NonSOLR_unbalanced_v2.csv')['TIC'].astype(int).tolist()

# === Loop through files in folder1 and copy matching TIC files ===
for idx, file_name in enumerate(os.listdir(source_dir)):
    if file_name.endswith(".csv") and not file_name.startswith("."):
        tic_id = int(file_name.split('_')[0])
        if tic_id in tics:
            #print(f"Processing file: {file_name} (Index: {idx})")
            source_path = os.path.join(source_dir, file_name)
            dest_path = os.path.join(destination_dir, file_name)
            if(os.path.exists(dest_path)):
                print(f"File {file_name} already exists in destination. Skipping copy.")
                continue
            shutil.copy(source_path, dest_path)
            print(f"Copied: {file_name}")


File 101110659_missing_lcp.csv already exists in destination. Skipping copy.
File 101225771_missing_lcp.csv already exists in destination. Skipping copy.
File 101225782_missing_lcp.csv already exists in destination. Skipping copy.
File 101282171_missing_lcp.csv already exists in destination. Skipping copy.
File 101315506_missing_lcp.csv already exists in destination. Skipping copy.
File 101315982_missing_lcp.csv already exists in destination. Skipping copy.
File 101922603_missing_lcp.csv already exists in destination. Skipping copy.
File 101961787_missing_lcp.csv already exists in destination. Skipping copy.
File 102054241_missing_lcp.csv already exists in destination. Skipping copy.
File 1028186_missing_lcp.csv already exists in destination. Skipping copy.
File 102916144_missing_lcp.csv already exists in destination. Skipping copy.
File 102982821_missing_lcp.csv already exists in destination. Skipping copy.
File 103057885_missing_lcp.csv already exists in destination. Skipping copy.
F

KeyboardInterrupt: 